In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip "/content/drive/My Drive/Data-sets/audio.zip"

Archive:  /content/drive/My Drive/Data-sets/audio.zip
replace __MACOSX/._audio? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio

In [6]:
DATASET_ROOT = "/content/audio"

AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"

DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

In [7]:
VALID_SPLIT = 0.1

SHUFFLE_SEED = 43

SAMPLING_RATE = 16000

SCALE = 0.5

BATCH_SIZE = 128

EPOCHS = 10

Pre-processing DataSet 

In [8]:
# If folder audio, does not exist, create it, otherwise do nothing
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)

# If folder noise, does not exist, create it, otherwise do nothing
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)

for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            # If folder is audio or noise, do nothing
            continue
        elif folder in ["other", "_background_noise_"]:
            # If folder is one of the folders that contains noise samples move it to the noise folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            # Otherwise, it should be a speaker folder, then move it to audio folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

Noise 

In [9]:
# Get the list of all noise files
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

print("Found {} files belonging to {} directories".format(len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))))

Found 6 files belonging to 2 directories


In [10]:
command = (
    "for dir in `ls -1 " + DATASET_NOISE_PATH + "`; do "
    "for file in `ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

# Split noise into chunks of 16,000 steps each
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == SAMPLING_RATE:
        # Number of slices of 16000 each that can be generated from the noise sample
        slices = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: slices * SAMPLING_RATE], slices)
        return sample
    else:
        print("Sampling rate for {} is incorrect. Ignoring it".format(path))
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

print(
    "{} noise files were split into {} noise samples where each is {} sec. long".format(
        len(noise_paths), noises.shape[0], noises.shape[1] // SAMPLING_RATE
    )
)

6 noise files were split into 354 noise samples where each is 1 sec. long


Data-set Gen

In [11]:

def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        # Create a random tensor of the same size as audio ranging from
        # 0 to the number of noise stream samples that we have.
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        # Get the amplitude proportion between the audio and the noise
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        # Adding the rescaled noise to audio
        audio = audio + noise * prop * scale

    return audio


def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1]), :])


# Get the list of audio file paths along with their corresponding labels

class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Processing speaker {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names))
)

# Shuffle
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

# Split into training and validation
num_val_samples = int(VALID_SPLIT * len(audio_paths))
print("Using {} files for training.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Using {} files for validation.".format(num_val_samples))
valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

# Create 2 datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)


# Add noise to the training set
# train_ds = train_ds.map(
#     lambda x, y: (add_noise(x, noises, scale=SCALE), y),
#     num_parallel_calls=tf.data.experimental.AUTOTUNE,
# )

# Transform audio wave to the frequency domain using `audio_to_fft`
# train_ds = train_ds.map(
#     lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
# )


train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))
print("--------")
print(train_ds)

# valid_ds = valid_ds.map(
#     lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
# )
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

Our class names: ['Nelson_Mandela', 'harshit', 'avish']
Processing speaker Nelson_Mandela
Processing speaker harshit
Processing speaker avish
Found 1800 files belonging to 3 classes.
Using 1620 files for training.
Using 180 files for validation.
--------
<MapDataset shapes: ((None, 16000, 1), (None,)), types: (tf.float32, tf.int32)>


Model

In [12]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE, 1), len(class_names))

model.summary()

# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 16000, 1)]   0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 16000, 16)    64          input[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 16000, 16)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 16000, 16)    784         activation[0][0]                 
_______________________________________________________________________________________

Training

In [13]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/10
13/13 [==============================] - 4s 283ms/step - loss: 0.6805 - accuracy: 0.5802 - val_loss: 0.5610 - val_accuracy: 0.7667
Epoch 2/10
13/13 [==============================] - 3s 200ms/step - loss: 0.6740 - accuracy: 0.7019 - val_loss: 0.4463 - val_accuracy: 0.6889
Epoch 3/10
13/13 [==============================] - 3s 199ms/step - loss: 0.4507 - accuracy: 0.7198 - val_loss: 0.4650 - val_accuracy: 0.6944
Epoch 4/10
13/13 [==============================] - 3s 218ms/step - loss: 0.3638 - accuracy: 0.8321 - val_loss: 0.2057 - val_accuracy: 0.9000
Epoch 5/10
13/13 [==============================] - 3s 198ms/step - loss: 0.3314 - accuracy: 0.8870 - val_loss: 0.2923 - val_accuracy: 0.7111
Epoch 6/10
13/13 [==============================] - 3s 197ms/step - loss: 0.3647 - accuracy: 0.8111 - val_loss: 0.2069 - val_accuracy: 0.9000
Epoch 7/10
13/13 [==============================] - 3s 216ms/step - loss: 0.2770 - accuracy: 0.9000 - val_loss: 0.3122 - val_accuracy: 0.9278
Epoch 

Accuracy

In [14]:
print(model.evaluate(valid_ds))

6/6 [==============================] - 0s 18ms/step - loss: 0.1014 - accuracy: 0.9722
[0.10144060850143433, 0.9722222089767456]


In [15]:
print(valid_ds)

<MapDataset shapes: ((None, 16000, 1), (None,)), types: (tf.float32, tf.int32)>


Predict

In [16]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

print(test_ds)
print(valid_ds)

for audios, labels in test_ds.take(1):
    # Get the signal FFT
    ffts = audio_to_fft(audios)
    # Predict
    y_pred = model.predict(audios)
    # Take random samples
    rnd = np.random.randint(0, BATCH_SIZE, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        # For every sample, print the true and predicted label
        # as well as run the voice with the noise
        # print(model.evaluate(test_ds)[1])
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        if labels[index] ==y_pred[index] and model.evaluate(test_ds)[1] > 0.90:
            # print(model.evaluate(test_ds)[1])
            print(y_pred[index])
            print("Welcome")
        else:
            print("Sorry")
        # print("The speaker is" if labels[index] == y_pred[index] else "", class_names[y_pred[index]])

<MapDataset shapes: ((None, 16000, 1), (None,)), types: (tf.float32, tf.int32)>
<MapDataset shapes: ((None, 16000, 1), (None,)), types: (tf.float32, tf.int32)>
Speaker: Nelson_Mandela	Predicted: Nelson_Mandela
2/2 [==============================] - 0s 62ms/step - loss: 0.1006 - accuracy: 0.9778
0
Welcome
Speaker: avish	Predicted: avish
2/2 [==============================] - 0s 20ms/step - loss: 0.1115 - accuracy: 0.9667
2
Welcome
Speaker: harshit	Predicted: harshit
2/2 [==============================] - 0s 20ms/step - loss: 0.0954 - accuracy: 0.9833
1
Welcome
Speaker: Nelson_Mandela	Predicted: Nelson_Mandela
2/2 [==============================] - 0s 20ms/step - loss: 0.0941 - accuracy: 0.9833
0
Welcome
Speaker: harshit	Predicted: harshit
2/2 [==============================] - 0s 20ms/step - loss: 0.0966 - accuracy: 0.9722
1
Welcome
Speaker: harshit	Predicted: harshit
2/2 [==============================] - 0s 20ms/step - loss: 0.1508 - accuracy: 0.9722
1
Welcome
Speaker: harshit	Predict

In [ ]:
#Predcit the speaker from the test dataset for real time pred. 

In [19]:
def predict(path, labels):
	test = paths_and_labels_to_dataset(path, labels)


	test = test.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
		BATCH_SIZE
	)

	test = test.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

	print(test)


	for audios, labels in test.take(1):
		# Get the signal FFT
		# ffts = audio_to_fft(audios)
		# Predict
		y_pred = model.predict(audios)
		# Take random samples
		rnd = np.random.randint(0, 1, 1)
		audios = audios.numpy()[rnd, :]
		labels = labels.numpy()[rnd]
		y_pred = np.argmax(y_pred, axis=-1)[rnd]

		for index in range(1):
			# For every sample, print the true and predicted label
			# as well as run the voice with the noise
			print(
				"\033[31m[*]\033[0m Predicted:\33{} {}\33[0m".format(
					"[92m",y_pred[index]
				)
			)
			if y_pred[index] == 0 and model.evaluate(valid_ds)[1] > 0.95:
				print("\033[31m[*]\033[0m Welcome user 0")
			elif y_pred[index] == 1 and model.evaluate(valid_ds)[1] > 0.95:
				print("\033[31m[*]\033[0m Welcome user 1")
			elif y_pred[index] == 2 and model.evaluate(valid_ds)[1] > 0.95:
				print("\033[31m[*]\033[0m Welcome user 2")
			elif y_pred[index] == 3 and model.evaluate(valid_ds)[1] > 0.95:
				print("\033[31m[*]\033[0m Welcome user 3")
			else:
				print("New user")



path = ["/content/244.wav"]
print(path)
labels = ["unknown"]

# path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
# audio_ds = path_ds.map(lambda x: path_to_audio(x))
# label_ds = tf.data.Dataset.from_tensor_slices(labels)
# return tf.data.Dataset.zip((audio_ds, label_ds))

predict(path, labels)

['/content/244.wav']
<MapDataset shapes: ((None, 16000, 1), (None,)), types: (tf.float32, tf.string)>
[*] Predicted: 1
6/6 [==============================] - 0s 19ms/step - loss: 0.1289 - accuracy: 0.9667
[*] Welcome user 1
